In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_curve, roc_auc_score

## Load the data

In [ ]:
train_data_2008 = np.loadtxt('data/train_2008.csv', skiprows=1, delimiter=',')
X_test_2008 = np.loadtxt('data/test_2008.csv', skiprows=1, delimiter=',')

In [ ]:
X_train_2008 = train_data_2008[:,:-1]
Y_train_2008 = train_data_2008[:,-1]

## Data pre-process

In [ ]:
def normalize_data_column(x):
    '''
    normalize the input data such that it is centered around zero and has standard deviation of 1.0
    Inputs:
        x: a (N, D) shaped numpy array containing the data points.
    Outputs:
        xp: a (N, D) shaped numpy array containing the normalized data points.
    '''
    xp = np.zeros_like(x)
    
    for idx_D in range(len(x[0,:])): #normalize each column independently
        average = np.mean(x[:,idx_D])
        std_dev = np.std(x[:,idx_D])
        if std_dev > 0:
            xp[:,idx_D] = (x[:, idx_D] - average)/std_dev
        elif average != 0: #if all the elements are the same in that column, make all of them to be one
            xp[:,idx_D] = x[:, idx_D]/average
        else:
            xp[:,idx_D] = x[:, idx_D]
    
    return xp

In [ ]:
#normalize each column of the data
X_train_2008 = normalize_data_column(X_train_2008)
X_test_2008 = normalize_data_column(X_test_2008)

In [ ]:
#split the training data into training and validation dataset
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train_2008, Y_train_2008, test_size=0.3)

## Build and train the model

In [ ]:
#estimator for adaboost
ada_tree_estimator = DecisionTreeRegressor(max_features='sqrt', splitter='random', min_samples_split=4, max_depth=3)
#adaboost regressor
ab = AdaBoostRegressor(ada_tree_estimator, learning_rate=0.01, loss='square', n_estimators=1000)
#fit
ab.fit(X_train, Y_train)

## Validation

In [ ]:
def visualize_loss_curves(model, X_train, Y_train, X_test, Y_test):
    plt.close('all')

    Y_train_pred = np.zeros_like(Y_train)
    Y_test_pred = np.zeros_like(Y_test)
    losses_train = []
    losses_test = []

    # For each added classifier, store the new training and test losses.
    for clf in model.estimators_:
        Y_train_pred += clf.predict(X_train)
        Y_test_pred += clf.predict(X_test)

        losses_train.append(len(np.where(np.sign(Y_train_pred-0.5) != np.sign(Y_train-0.5))[0]) / len(Y_train_pred))
        losses_test.append(len(np.where(np.sign(Y_test_pred-0.5) != np.sign(Y_test-0.5))[0]) / len(Y_test_pred))

    # Plot the losses across n_clfs.
    n_clfs = len(model.estimators_)
    plt.plot(np.arange(1, n_clfs + 1), losses_train)
    plt.plot(np.arange(1, n_clfs + 1), losses_test)
    plt.title('Loss vs. n_clfs')
    plt.legend(['Training loss', 'Test loss'])
    plt.xlabel('n_clfs')
    plt.ylabel('Loss')
    plt.show()

In [ ]:
visualize_loss_curves(ab, X_train, Y_train, X_valid, Y_valid)

In [ ]:
## draw the ROC curve
Y_valid_predict = ab.predict(X_valid)
fpr, tpr, _ = roc_curve(Y_valid, Y_valid_predict)
plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Background (false positives)')
plt.ylabel('Signal (true positives)')
plt.title(r'ROC curve')

In [ ]:
## calculate the AUC - area under the curve
AUC = roc_auc_score(Y_valid, Y_valid_predict)
print(AUC)

## Make the prediction!

In [ ]:
predict_test_2008 = ab.predict(X_test_2008)

## Write to submission file

In [ ]:
ids_2008 = np.arange(len(X_test_2008))
target_2008 = np.copy(predict_test_2008)

In [ ]:
subm_2008 = np.stack([ids_2008,target_2008], axis=1)
#let's name the submission file in this way
#submission_2008or2012_date_version_algorithm_person.csv
np.savetxt('submission/submission_2008_02092019_v1_Adaboost_Zhicai.csv', subm_2008, fmt='%d,%.6f', header='id,target', comments='')